In [ ]:
# default_exp runner

# Runner

> A function to run alphapept.

In [ ]:
#hide
from nbdev.showdoc import *

ToDo:
* Multiple file handling
* Parallel processing

In [ ]:
#export
import logging

def check_enviroment():
    import numba 
    if float('.'.join(numba.__version__.split('.')[0:2])) < 0.46:
        raise RuntimeError('Numba version {} not sufficient'.format(numba.__version__))

def check_settings(settings):

    """
    Consistency check for settings.

    """
    check_enviroment()
    
    logging.info('Checking raw path.')

    #Check if a valid raw file is provided. If a npz file is also provided do not convert

    if os.path.isfile(settings["raw"]["raw_path_npz"]):
        if settings["general"]["convert_raw"]:
            settings["general"]["convert_raw"] = False
            logging.info('NPZ for raw file present. Skipping conversion step.')
    else:
        if os.path.isfile(settings["raw"]["raw_path"]):
            logging.info('No NPZ for raw present. Performing conversion step.')
            settings["general"]["convert_raw"] = True
        else:
            raise FileNotFoundError('No raw or converted raw file provided')
            
            
    #check filetype -> todo what if we have an npz but no irignal file
    
    base, ext = os.path.splitext(settings["raw"]["raw_path"])
    
    if ext == '.raw':
        datatype='thermo'
    elif ext == '.d':
        datatype='bruker'
    else:
        raise NotImplementedError('File extension {} not understood.'.format(ext))

    logging.info('Raw path okay.')

    #Check library file
    logging.info('Checking library path.')

    if os.path.isfile(settings["fasta"]["library_path"]):
        if settings["general"]["create_library"]:
            settings["general"]["create_library"] = False
            logging.info('NPZ for library file present. Skipping library creation step.')
    else:
        if os.path.isfile(settings["fasta"]["fasta_path"]):
            logging.info('No NPZ for library present. Creating library from FASTA.')
            settings["general"]["create_library"] = True
        else:
            raise FileNotFoundError('No FASTA or library file provided')

    logging.info('Library path okay.')
    
    
    settings['general']['type'] = datatype

    return settings

In [ ]:
#export
import numpy as np
import os
from functools import partial
import pandas as pd

#alpha_runner(self.settings, self.update_global_progress, self.update_current_progress, self.update_task)

def alpha_runner(settings, overall_progress = None, current_progress = None, CURRENT_TASK = print):
    """
    A wrapper function to run alphapept
    """
    
    #assumes the dict is ordered 

    time_dict = {}

    timings = [10,15,20,20,20,15]
    total_time = 0
    for idx, element in enumerate(['library_generation', 'file_conversion', 'feature_finding', 'search', 'combining_files', 'reporting']):
        time_dict[element] = {}
        time_dict[element]['step'] = timings[idx]

        total_time += timings[idx]
        time_dict[element]['total'] = total_time


    def check_time_dict(time_dict):
        """
        Function to check the validity of a time dict
        """

        total_time = 0

        for key in time_dict.keys():
            total_time += time_dict[key]['step']

        assert total_time == 100

    check_time_dict(time_dict)
    
    progress = 0

    def progress_wrapper(current, delta=1):
        """
        Wrapper function to change the overall progress with the current progress
        """

        global progress
        
        if current_progress:
            current_progress(current)
        
        overall = progress + current*delta
        
        if overall_progress:
            overall_progress(overall)

        if current == 1.0:
            progress += delta

    def set_progress(overall):
        global progress

        overall_progress(overall)

        progress = overall


    from alphapept.constants import mass_dict

    CURRENT_TASK('Checking Settings')
    settings = check_settings(settings)


    ### Library Generation ###

    if settings["general"]["create_library"]:
        from alphapept.fasta import generate_library, generate_spectra, save_library

        te = time_dict['library_generation']['step']
        n_steps = 2

        CURRENT_TASK('Digesting FASTA')
        to_add, pept_dict, fasta_dict = generate_library(mass_dict, callback = partial(progress_wrapper, delta=1/n_steps*te), **settings['fasta'])
        logging.info('Digested {:,} proteins and generated {:,} peptides'.format(len(fasta_dict), len(to_add)))

        CURRENT_TASK('Generating Spectra')
        spectra = generate_spectra(to_add, mass_dict, callback = partial(progress_wrapper, delta=1/n_steps*te))
        logging.info('Generated {:,} spectra'.format(len(spectra)))

        CURRENT_TASK('Saving library')
        base, ext = os.path.splitext(settings['fasta']['fasta_path'])
        settings['fasta']['library_path'] = base + '.npz'
        library_path = save_library(spectra, pept_dict, fasta_dict, **settings['fasta'])             
        logging.info('Database saved to {}. Filesize {:.2f} Gb'.format(library_path, os.stat(library_path).st_size/(1024**3)))

    set_progress(time_dict['library_generation']['total'])
    ### File Conversion ###

    if settings["general"]["convert_raw"]:

        te = time_dict['file_conversion']['step']

        from alphapept.io import raw_to_npz

        CURRENT_TASK('Converting raw files')
        out_path = raw_to_npz(settings["raw"], callback=partial(progress_wrapper, delta=te))
        settings["raw"]["query_path"] = out_path[0]
        logging.info('Raw file(s) saved to {}'.format(out_path))
    else:
        settings["raw"]["query_path"] = settings["raw"]["raw_path_npz"]

    db_data = np.load(settings["fasta"]["library_path"], allow_pickle=True)
    query_data = np.load(settings["raw"]["query_path"], allow_pickle=True)


    set_progress(time_dict['file_conversion']['total'])

    if settings["general"]["find_features"]:

        te = time_dict['feature_finding']['step']

        logging.info('Finding Features')

        if settings["general"]['type'] == 'thermo':

            n_steps = 8

            logging.info('Fature Finding for Thermo')
            #Feature Finding Part

            from alphapept.feature_finding import raw_to_centroid, get_hills, split_hills, filter_hills, get_hill_data, get_edges, get_isotope_patterns, feature_finder_report
            from alphapept.constants import averagine_aa, isotopes


            CURRENT_TASK('Converting centroids')
            centroids = raw_to_centroid(query_data, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Loaded {:,} centroids.'.format(len(centroids)))

            CURRENT_TASK('Exctracting hills')
            completed_hills = get_hills(centroids, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('A total of {:,} hills extracted. Average hill length {:.2f}'.format(len(completed_hills), np.mean([len(_) for _ in completed_hills])))

            CURRENT_TASK('Splitting hills')
            splitted_hills = split_hills(completed_hills, centroids, smoothing=1, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Split {:,} hills into {:,} hills'.format(len(completed_hills), len(splitted_hills)))

            CURRENT_TASK('Refining hills')
            filtered_hills = filter_hills(splitted_hills, centroids, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Filtered {:,} hills. Remaining {:,} hills'.format(len(splitted_hills), len(filtered_hills)))

            CURRENT_TASK('Calculating hill statistics')
            sorted_hills, sorted_stats, sorted_data = get_hill_data(filtered_hills, centroids, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Extracting hill stats complete')

            CURRENT_TASK('Connecting pre isotope patterns')
            pre_isotope_patterns = get_edges(sorted_stats, sorted_data, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Found {} pre isotope patterns.'.format(len(pre_isotope_patterns)))

            CURRENT_TASK('Deisotope patterns')
            isotope_patterns, isotope_charges = get_isotope_patterns(pre_isotope_patterns, sorted_stats, sorted_data, averagine_aa, isotopes, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Extracted {} isotope patterns.'.format(len(isotope_patterns)))

            CURRENT_TASK('Calculating feature statistics')
            feature_table = feature_finder_report(isotope_patterns, isotope_charges, sorted_stats, sorted_data, sorted_hills, query_data, callback=partial(progress_wrapper, delta=te*1/n_steps))
            logging.info('Report complete.')

        elif settings["general"]['type'] == 'bruker':

            logging.info('Fature Finding for Bruker')
            from alphapept.feature_finding import extract_bruker, convert_bruker

            feature_path = extract_bruker(settings['raw']['raw_path'])
            feature_table = convert_bruker(feature_path)

            logging.info('Extracted {:,} features.'.format(len(feature_table)))

        else:
            raise NotImplementedError('Filetype {} not understood.'.format(settings["general"]["type"]))

        from alphapept.feature_finding import map_ms2
        features = map_ms2(feature_table, query_data)

    else:
        logging.info('Using Features from raw data')
        from alphapept.search import query_data_to_features
        features = query_data_to_features(query_data)


    set_progress(time_dict['feature_finding']['total'])

    ### Search part ###

    te = time_dict['search']['step']
    n_steps = 2

    from alphapept.search import get_psms, get_score_columns
    from alphapept.score import score_x_tandem

    CURRENT_TASK('Running first search.')
    psms, num_specs_compared = get_psms(query_data, db_data, features, callback=partial(progress_wrapper, delta=te*1/n_steps), **settings["search"])
    logging.info('First search complete. Compared {:,} spectra and found {:,} psms.'.format(num_specs_compared, len(psms)))

    CURRENT_TASK('Extracting columns for scoring.')
    psms, num_specs_scored = get_score_columns(psms, query_data, db_data, features, **settings["search"])
    logging.info('Extracted columns for {:,} psms.'.format(num_specs_scored))


    if settings['search']['calibrate']:

        CURRENT_TASK('Scoring psms.')
        df = score_x_tandem(pd.DataFrame(psms), plot=False, verbose=False, **settings["search"])
        logging.info('Scoring complete. For {} FDR found {:,} targets and {:,} decoys.'.format(settings["search"]["peptide_fdr"], df['target'].sum(), df['decoy'].sum()) )


        from alphapept.recalibration import get_calibration

        CURRENT_TASK('Calibrating features.')
        logging.info('Precursor Offset (PPM) is {:.2f} (mean), {:.2f} (std)'.format(df['o_mass_ppm'].mean(), df['o_mass_ppm'].std()))
        'Calibrating MS1 spectra'
        features_calib, df_sub = get_calibration(df, features, **settings["calibration"])

        o_mass_ppm_mean = df_sub['o_mass_ppm_calib'].mean()
        o_mass_ppm_std = df_sub['o_mass_ppm_calib'].std()

        logging.info('Calibration complete. Precursor Offset (PPM) is {:.2f} (mean), {:.2f} (std)'.format(o_mass_ppm_mean, o_mass_ppm_std))

        logging.info('Adjusting search bound to {:.2f} ppm.'.format(3*o_mass_ppm_std))

        settings["search"]["m_offset_calibrated"] = float(3*o_mass_ppm_std)

        CURRENT_TASK('Running second search.')
        psms, num_specs_compared = get_psms(query_data, db_data, features_calib, callback=partial(progress_wrapper, delta=te*1/n_steps), **settings["search"])
        logging.info('Second search complete. Compared {:,} spectra and found {:,} psms.'.format(num_specs_compared, len(psms)))

        CURRENT_TASK('Extracting columns for scoring.')
        psms, num_specs_scored = get_score_columns(psms, query_data, db_data, features_calib, **settings["search"])
        logging.info('Extracted columns for {:,} psms.'.format(num_specs_scored))

    ## Protein Groups and FDR control


    set_progress(time_dict['search']['total'])

    set_progress(time_dict['combining_files']['total'])


    from alphapept.score import cut_global_fdr, perform_protein_grouping, cut_global_fdr, get_x_tandem_score, filter_score

    CURRENT_TASK('Scoring')
    df = pd.DataFrame(psms)
    df['score'] = get_x_tandem_score(df)
    df['decoy'] = df['sequence'].str[-1].str.islower()
    df = filter_score(df)

    CURRENT_TASK('FDR control on peptides')
    df = cut_global_fdr(df, analyte_level='sequence',  plot=False, verbose=False)
    logging.info('Scoring peptides complete. For {} FDR found {:,} targets and {:,} decoys.'.format(settings["search"]["peptide_fdr"], df['target'].sum(), df['decoy'].sum()) )


    CURRENT_TASK('Perform protein grouping')
    df = perform_protein_grouping(df, db_data['pept_dict'].item(), db_data['fasta_dict'].item())
    df = cut_global_fdr(df, analyte_level='protein',  plot=False, verbose=False)
    logging.info('Scoring proteins complete. For {} FDR found {:,} targets and {:,} decoys. A total of {:,} proteins found.'.format(settings["search"]["protein_fdr"], df['target'].sum(), df['decoy'].sum(), len(set(df['protein']))))

    ## Quantification
    if settings["quantification"]["max_lfq"]:
        raise NotImplementedError("MaxLFQ not implemented yet.")
        
    ## Misc
    if settings["misc"]["match_between_runs"]:
        raise NotImplementedError("Match beween runs not implemented yet.")
        
    CURRENT_TASK('Saving.')

    base, ext = os.path.splitext(settings['raw']['query_path'])
    out_path = base+'_ap.csv'
    df.to_csv(out_path, index = False)
    logging.info('Saved to {}'.format(out_path))

    import yaml

    out_path_settings = base+'_ap.yaml'

    with open(out_path_settings, 'w') as file:
        yaml.dump(settings, file)

    logging.info('Settings saved to {}'.format(out_path_settings))

    set_progress(time_dict['reporting']['total'])

    CURRENT_TASK('Complete.')
    
    return df, features

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_settings.ipynb.
Converted 13_runner.ipynb.
Converted index.ipynb.
